In [ ]:
import time
from tqdm import tqdm
from scrapy.crawler import CrawlerProcess
import pandas as pd
import numpy as np
from collections import Counter
from scraping import *

In [ ]:
do_scraping = False

In [ ]:
if do_scraping:
    pop_df = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/most_pop_nodes.csv")
    N = len(pop_df)#50
    nodes = list(pop_df["index"][:N])

nodes = [
            '03bb88ccc444534da7b5b64b4f7b15e1eccb18e102db0e400d4b9cfe93763aa26d'
        ]

In [ ]:
if do_scraping:
    #spider = LNNodeParser(nodes=nodes)
    process = CrawlerProcess(LNNodeParser)
    process.crawl()
    process.start()

In [ ]:
if do_scraping:
    process.stop()

In [ ]:
if do_scraping:
    meta_records = []
    for i, n in enumerate(nodes):
        meta_records.append(extract_node_meta_data(n))
        print(i)
    meta_df = pd.DataFrame(meta_records)
    print(meta_df.head())
    print(len(meta_records))
    print((meta_df.isnull().sum() / N).sort_values())
    meta_df.to_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_meta.csv", index=False)
else:
    meta_df = pd.read_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_meta.csv")

In [ ]:
meta_with_labels_df = meta_df[~meta_df["labels"].isnull()]
if not do_scraping:
    meta_with_labels_df["labels"] = meta_with_labels_df["labels"].apply(eval)

In [ ]:
meta_with_labels_df.head(3)

### Label merging ideas:
  
- G1: every kind of games
- G2: Banking, Excange (not G1)
- G3: Wallet (not G1 or G2)
- G4: every kind of Store (not G1, G2, G3)
- G5: all remaining service providers (not G1, G2, G3, G4)

Ami csak 'Open Source' vagy 'Testing' azokat nem venném figyelembe

In [ ]:
all_labels = []
for lab_list in list(meta_with_labels_df["labels"]):
    all_labels += lab_list

In [ ]:
cnt = Counter(all_labels)

In [ ]:
len(cnt), len(meta_with_labels_df)

In [ ]:
cnt.most_common()

In [ ]:
label_keys = list(cnt.keys())

In [ ]:
for label in tqdm(label_keys):
    meta_with_labels_df[label] = meta_with_labels_df["labels"].apply(lambda x: 1 if label in x else 0)

In [ ]:
meta_with_labels_df.drop(['Open Source', 'Testing'], axis=1, inplace=True)
label_keys.remove('Open Source')
label_keys.remove('Testing')

In [ ]:
meta_with_labels_df["num_valid_labels"] = meta_with_labels_df[label_keys].sum(axis=1)

In [ ]:
meta_with_labels_df["num_valid_labels"].value_counts()

In [ ]:
meta_with_valid_labels = meta_with_labels_df[meta_with_labels_df["num_valid_labels"]>0]

In [ ]:
print(len(meta_with_valid_labels))

In [ ]:
meta_with_valid_labels.to_csv("/mnt/idms/fberes/data/bitcoin_ln_research/node_meta_with_labels.csv", index=False)